In [1]:
import sys
quic_path = '/home/quic/QUIC-Projects'
if not quic_path in sys.path:
    sys.path.append(quic_path)

In [2]:
import numpy as np 
from matplotlib import pyplot as plt

In [3]:
# for SVM
from classifiers.convex.svm import BinarySVM
from classifiers.kernel import Kernel
from classifiers.datasets.dataloader import Example_4x2
# for QASVM
from classifiers.quantum import Qasvm_Mapping_4x2
from classifiers.quantum.qasvm import QASVM
from classifiers.optimizer import SPSA
from qiskit.aqua import QuantumInstance
from qiskit.ignis.mitigation import CompleteMeasFitter
from qiskit.circuit.library import RealAmplitudes, EfficientSU2, NLocal, TwoLocal
from qiskit.providers.aer import QasmSimulator
from qiskit import IBMQ
# for logging & visualization
import logging
from classifiers.callback import CostParamStorage, ParamsStopping
from torch.utils.tensorboard import SummaryWriter
from classifiers.visualization import Plot_Data, Plot_Log_From_SPSA, Plot_SVM, compare_svm_and_qasvm

writer = SummaryWriter()
storage = CostParamStorage(writer)

if IBMQ.active_account() is None:
    IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-kaist', group='internal', project='default')
backend = provider.get_backend('ibmq_toronto')
backend

<IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-kaist', group='internal', project='default')>

In [4]:
X, y = Example_4x2(True)()

In [5]:
var_form = RealAmplitudes(2, reps=1)
quantum_instance = QuantumInstance(backend=QasmSimulator.from_backend(backend), shots=2**13, basis_gates=backend.configuration().basis_gates, initial_layout=Qasvm_Mapping_4x2(backend).layout, measurement_error_mitigation_cls=CompleteMeasFitter, optimization_level=2, seed_transpiler=0)
qasvm = QASVM(X, y, quantum_instance=quantum_instance, C=1, k=0.1, option='Bloch_sphere', var_form=var_form).dual
print(qasvm.second_order_circuit.depth())
qasvm.second_order_circuit.draw(fold=-1, idle_wires=False)

72


global phase: 5π/8
              ┌────┐  ┌────────────────┐   ┌────┐  ┌────────┐     ┌───┐     ┌──────────┐┌────┐   ┌─────────┐                                                                                                                                                                                                                                                                                                                                                         ┌───┐                                                ┌───┐           ┌───┐                                                                                                                                                                   ░          
  i_0 -> 5 ───┤ √X ├──┤ RZ(θ_i[0] + π) ├───┤ √X ├──┤ RZ(3π) ├──■──┤ X ├──■──┤ RZ(3π/2) ├┤ √X ├───┤ RZ(π/2) ├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├────────────────────────────────────────────────┤ X ├─────■─────┤ X ├──────■─────────────────────────■──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────░──────────
              ├────┤  └────────────────┘   └────┘  └────────┘┌─┴─┐└─┬─┘┌─┴─┐└──────────┘├────┤┌──┴─────────┴───┐      ┌────┐      ┌────────┐                      ┌───┐                                                                                  ┌───┐             ┌───┐  ┌────────┐  ┌────┐  ┌────────────┐    ┌────┐    ┌───┐ ┌─────────────┐     ┌───┐     ┌─────────────┐     ┌───┐     ┌───────────────┐      ┌───┐           ┌───┐                     └─┬─┘           ┌───┐     ┌───┐     ┌───┐            └─┬─┘   ┌─┴─┐   └─┬─┘    ┌─┴─┐    ┌───┐┌──────────┐┌─┴─┐┌─────────┐┌────┐┌──────────┐                                                                                                   ░    ┌─┐   
 yi_0 -> 8 ───┤ √X ├─────────────────────────────────────────┤ X ├──■──┤ X ├─────■──────┤ √X ├┤ RZ(θ_i[2] + π) ├──────┤ √X ├──────┤ RZ(3π) ├────■─────────■───────┤ X ├─────────■───────────────────────■────────────────────────────────────────■───────┤ X ├───────■─────┤ X ├──┤ RZ(-π) ├──┤ √X ├──┤ RZ(3.1454) ├────┤ √X ├────┤ X ├─┤ RZ(0.62028) ├─────┤ X ├─────┤ RZ(-1.6377) ├─────┤ X ├─────┤ RZ(-0.014047) ├──────┤ X ├────────■──┤ X ├───────■───────────────■──────────■──┤ X ├──■──┤ X ├──■──┤ X ├─────■────────■─────┤ X ├─────■──────┤ X ├────┤ X ├┤ RZ(3π/2) ├┤ X ├┤ RZ(π/2) ├┤ √X ├┤ RZ(3π/2) ├───────────────────────────────────────────────────────────────────────────────────────────────────░────┤M├───
              ├────┤  ┌────────────────┐   ┌────┐  ┌────────┐└───┘     └───┘   ┌─┴─┐    ├────┤├────────────────┤      ├────┤      ├────────┤    │       ┌─┴─┐     └─┬─┘       ┌─┴─┐                     │                                        │       └─┬─┘       │     └─┬─┘  └────────┘  └────┘  └────────────┘    └────┘    └─┬─┘ └─────────────┘     └─┬─┘     └─────────────┘     └─┬─┘     └───────────────┘      └─┬─┘      ┌─┴─┐└─┬─┘     ┌─┴─┐      ┌──────────────┐┌─┴─┐└─┬─┘┌─┴─┐└─┬─┘  │  └─┬─┘     │              └───┘            └───┘    └─┬─┘└──────────┘└───┘└─────────┘└────┘└──────────┘                                                                                                   ░    └╥┘   
  i_1 -> 9 ───┤ √X ├──┤ RZ(θ_i[1] + π) ├───┤ √X ├──┤ RZ(3π) ├──────────────────┤ X ├────┤ √X ├┤ RZ(θ_i[3] + π) ├──────┤ √X ├──────┤ RZ(3π) ├────┼───────┤ X ├───────■─────────┤ X ├─────────────────────┼────────────────────────────────────────┼─────────┼─────────┼───────■──────────────────────────────────────────────────────┼─────────────────────────■─────────────────────────────┼────────────────────────────────■────────┤ X ├──■───────┤ X ├──────┤ RZ(0.027625) ├┤ X ├──

In [6]:
from tqdm.notebook import tqdm
epochs = 2**7
last_avg = 2**4

optimizer = SPSA(qasvm.cost_fn, qasvm.parameters)
optimizer.calibrate(epochs)

storage = CostParamStorage(writer)
early_stopping = ParamsStopping(patiance=0, last_avg=last_avg, tol=1e-2)

for epoch in tqdm(range(epochs)):
    optimizer.step(epoch)
    storage(epoch, qasvm.cost_fn, qasvm.parameters)
    if early_stopping(epoch, qasvm.parameters):
        pass
    else:
        qasvm.parameters = early_stopping.best_params
        break


TypeError: result() got an unexpected keyword argument 'wait'

In [ ]:
import qiskit
qiskit.__qiskit_version__

In [ ]:
!python --version

In [8]:
quantum_instance.is_simulator

True